In [ ]:
%matplotlib inline

import sys
import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

from lib.planar_patch_detector import PlanarPatchDetector
from lib.masked_yolo_detector import MaskedYOLODetector
from lib.utils import get_camera_intrinsics

In [ ]:
# Load depth image
depth_path = "../data/medium_box/raw_depth.png"
depth_image = cv.imread(depth_path, cv.IMREAD_ANYDEPTH)

if depth_image is None:
    raise FileNotFoundError(f"Failed to load depth image from {depth_path}")

# read camera intrinsics
intrinsics_path = "../data/small_box/intrinsics.json"
camera_intrinsics = get_camera_intrinsics(intrinsics_path)



In [ ]:
# run masked yolo detector
color_image = cv.imread("../data/small_box/color_image.png")
depth_image = cv.imread("../data/small_box/raw_depth.png", cv.IMREAD_ANYDEPTH)

yolo_detector = MaskedYOLODetector(
        color_image=color_image,
        depth_image=depth_image,
        model_path="/workspace/data/model/best.pt"
    )
yolo_detector.set_box_area(min_area=2000, max_area=10000)
yolo_detector.create_mask()
boxes = yolo_detector.run_detection(visualize=True)
yolo_detector.print_box_details()

In [ ]:
print(boxes)

In [ ]:
known_dims = {
        'small_box': [0.340, 0.250, 0.095],
        'medium_box': [0.255, 0.155, 0.100]
    }

# run planar patch detector
planar_detector = PlanarPatchDetector(
    color_img=color_image,
    depth_img=depth_image,
    intrinsics=camera_intrinsics,
    known_dims=known_dims
)

planar_detector.set_yolo_detections(boxes)
#run_interactive_slider(planar_detector)

# planar_detector.process_all(visualize=True,
#                             visualize_individual=True,
#                             mode='2d',
#                             normal_variance_threshold_deg=60,
#                             coplanarity_deg=60,
#                             min_plane_edge_length=0.03,
#                             min_num_points=20,
#                             knn=50
#                         )

planar_detector.process_all(
    visualize=False,
    visualize_individual=True,
    mode='2d',
    normal_variance_threshold_deg=60,
    coplanarity_deg=60,
    min_plane_edge_length=0.03,
    min_num_points=20,
    knn=50,
    voxel_size=0.01,  # Change voxel size
    distance_threshold=0.005,  # Change the RANSAC distance threshold
    nb_neighbors=30,  # Change number of neighbors for outlier removal
    std_ratio=1.5, # Adjust standard deviation ratio for outlier removal
    min_plane_area=0.00003  # Minimum area of the detected plane
)


# planar_detector.process_all(visualize=False, visualize_individual=False)

results = planar_detector.get_results()



In [ ]:
print(results)

In [ ]:
planar_detector.visualize_colored_surfaces()